**Imports**

PyTorch Geometric

In [1]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 5.1 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 5.0 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.5 MB 5.1 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 750 kB 5.4 MB/s 
     |████████████████████████████████| 407 kB 5.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=f4bd23d39cb5868bb051ce128a6dd39bd2c127fe5447849c58ce6790e9916d10
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [62]:
import pandas as pd
import numpy as np
import torch.nn.functional as F 

from torch_geometric.data import InMemoryDataset, download_url, Data
from datetime import datetime
from torch.nn import Linear
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

**Mount GDrive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
action_types.csv              participants_activity_anomalies.csv
activities.csv                patients.csv
activity_types.csv            scores.csv
anomalies.csv                 scores_predictions.csv
anomaly_types.csv             sensor_locations.csv
diagnosis_predictions.csv     sensors.csv
diagnosis_types.csv           tasks.csv
events.csv                    task_types.csv
overt_error_action_types.csv  trajectory_anomalies.csv


In [18]:
def task_activity_sensors(tasks, events, sensors, activities, patient):

  # Seleziona i task-activity eseguiti dal paziente 'patient'
  task_activity = tasks.query('patient == ' + patient)

  # Seleziona il tempo di inizio e fine delle attività svolte dal paziente 'patient'
  activities_patient = activities.query('patient == ' + patient)
  start_time = activities_patient.iloc[0]['start']
  end_time = activities_patient.iloc[len(activities_patient) - 1]['end']

  # Query nel dataframe events per selezionare solo i sensori utilizzati nelle attività
  sensors_events = events.query('patient == ' + patient + ' and time >= "' + start_time + '" and time <= "' + end_time + '"')

  # Matrice risultato che contiene i dati dei nodi
  res_node = pd.DataFrame(columns=['features'])
  res_edge = pd.DataFrame(columns=['times']) 

  # Dimensione dell'array delle features
  dim_features = len(sensors)

  # Dizionario che contiene la posizione del sensore nell'array delle features
  dict_sens = {}

  for i in range(0, dim_features):
    dict_sens[sensors.iloc[i]['sensor_id']] = i

  # Indice del task-activity
  i = 0

  # Numero edge del grafo
  num_edge = 0

  # Nome del task-activity precedente
  pair_str_prev = ""

  # Flag per salvataggio features di nodo e arco
  ins_upd = False
  last_node = False

  for i in range(0, len(task_activity)):

    # Controlla se il task-activity attuale è l'ultimo
    if ((i + 1) >= len(task_activity)):
      last_node = True

    # Nome del task-activity attuale
    pair_str = str(task_activity.iloc[i]['task']) + '-' + str(task_activity.iloc[i]['activity'])

    # Controlla se il task-activity attuale e quello precedente sono diversi
    if pair_str != pair_str_prev:

      # Viene salvato lo start time
      s_time = task_activity.iloc[i]['time']

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str

        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str
    
    else:
      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str
        
        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str

    # Controlla se start ed end time sono stati trovati
    if ins_upd:

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Aggiorna il numero di edge del grafo
        num_edge = num_edge + 1

        # Controlla se esiste un arco per il task-activity attuale 
        if pair_str in res_edge.index:

          # Aggiunge start ed end time all'arco
          res_edge.loc[pair_str, 'times'] += [((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]
        
        else:
          # Crea un nuovo arco
          res_edge.loc[pair_str] = [[((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]]

      # Controlla se non esiste un nodo per il task-activity attuale
      if not pair_str in res_node.index:

        # Crea un array vuoto per le fueatures del nodo
        features_node = np.zeros(dim_features, dtype=np.int8)

        # Seleziona i sensori utilizzati per il task-activity attuale
        task_activity_sensors = sensors_events.query('time >= "' + s_time + '" and time <= "' + e_time + '"')

        # Popola l'array delle features
        for _, s in task_activity_sensors.iterrows():
          features_node[dict_sens[s['sensor']]] = 1

        # Salva nel nodo l'array delle features
        res_node.loc[pair_str] = [features_node]
  
  return res_edge, res_node, num_edge

In [58]:
sensors = pd.read_csv('drive/MyDrive/CASAS400/sensors.csv')
events = pd.read_csv('drive/MyDrive/CASAS400/events.csv')
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
activities = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

patients = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')['patient'].unique()

dataset = []

for i in range(0, len(patients)):

  # Crea i dataframe con le features degli edges e dei nodi
  edge, node, num_edge = task_activity_sensors(tasks, events, sensors, activities, str(patients[i]))

  # Salva la label associata al paziente
  label = pd.read_csv('drive/MyDrive/CASAS400/patients.csv').query('patient_id == ' + str(patients[i]))['diagnosis'].values

  if len(node) > 0:
    atr_node = np.zeros([len(node), len(node.iloc[0]['features'])])

    # Converte il dataframe dei nodi in una matrice
    for j in range(0, len(node)):
      atr_node[j] = node.iloc[j]['features']

    adj_matrix = np.empty([num_edge, 2])
    index_adj_matrix = 0

    # Crea la matrice di addiacenza dei nodi
    for j in range(0, len(edge)):
      for k in range(0, len(edge.iloc[j]['times'])):
        f_node = node.index.get_loc(edge.iloc[j].name)
        s_node = node.index.get_loc(edge.iloc[j]['times'][k][1])

        adj_matrix[index_adj_matrix] = np.array([f_node, s_node])
        index_adj_matrix = index_adj_matrix + 1
    
    # Salva la mtrice delle features e la matrice di addiacenza del grafo
    dataset.append(Data(x=torch.tensor(atr_node, dtype=torch.float),
                        edge_index=torch.tensor(adj_matrix, dtype=torch.long).t().contiguous(),
                        y=torch.tensor(label, dtype=torch.float)))

88


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


117
184
13
85
67
171
132
127
211
136
111
275
185
301
194
52
255
210
228
225
181
172
280
251
314
48
18
338
183
299
286
393
161
331
139
229
230
25
325
368
129
253
14
249
208
335
108
137
295
358
55
252
256
110
138
53
37
140
30
154
311
8
147
236
354
259
182
350
336
242
366
263
177
50
270
233
76
400
205
20
220
152
54
103
221
216
279
153
232
342
273
289
387
39
105
318
180
40
238
10
231
326
33
367
203
372
142
215
163
188
170
200
93
70
371
283
82
264
167
244
113
261
35
373
102
327
312
276
319
91
186
356
146
282
394
271
173
56
81
4
365
323
83
274
15
192
191
388
383
165
164
28
193
352
386
128
90
281
202
375
6
197
363
45
241
155
267
247
362
141
115
212
22
382
47
169
101
340
99
120
234
321
41
391
266
250
243
42
351
72
222
361
84
395
175
360
46
359
376
277
9
346
378
308
118
389
143
313
63
145
310
223
133
130
158
38
79
107
349
237
214
71
59
29
330
293
339
100
69
58
341
174
294
106
305
51
95
268
298
399
297
77
290
24
309
392
121
254
5
149
347
144
26
98
337
306
134
248
80
32
317
112
87
384
304
11
348


In [71]:
pd.read_csv('drive/MyDrive/CASAS400/diagnosis_types.csv')

,diagnosis_id,description
0,1,dementia
1,2,MCI
2,3,middle age 45-59
3,4,young-old 60-74
4,5,old-old 75+
5,6,other medical
6,7,watch/at risk - follow longitudinally
7,8,younger adult
8,9,"younger adult, English second language"
9,10,diagnosis not available


In [106]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset[0].x.shape[1], 16)
        self.conv2 = GCNConv(16, 10)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (conv1): GCNConv(74, 16)
  (conv2): GCNConv(16, 10)
)
Number of parameters:  1370


In [83]:
embedding_size = 64

class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(dataset[0].x.shape[1], embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size*2, 1)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(74, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Number of parameters:  17409


In [108]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Wrap data in a data loader
data_size = len(dataset)
NUM_GRAPHS_PER_BATCH = 64
loader = DataLoader(dataset[:int(data_size * 0.8)], 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(dataset[int(data_size * 0.8):], 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

def train(data):
    # Enumerate over the data
    for batch in loader:
      # Use GPU
      batch.to(device)  
      # Reset gradients
      optimizer.zero_grad() 
      # Passing the node features and the connection info
      pred = model(batch) 
      # Calculating the loss and gradients
      loss = torch.sqrt(loss_fn(pred, batch.y))       
      loss.backward()  
      # Update using the gradients
      optimizer.step()   
    return loss

print("Starting training...")
losses = []
for epoch in range(2000):
    loss = train(dataset)
    losses.append(loss)
    if epoch % 100 == 0:
      print(f"Epoch {epoch} | Train Loss {loss}")

Starting training...
torch.Size([3582, 10])


ValueError: ignored

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv

data = dataset[0]


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(dataset[0].x.shape[1], 16, dim=1, kernel_size=2)
        self.conv2 = SplineConv(16, 10, dim=1, kernel_size=2)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    log_probs, accs = model(), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')